In [1]:
from pathlib import Path
import pandas as pd
import pickle

In [2]:
with open("/home/cherniak/itmo_job/GAMLET/data/meta_features_and_fedot_pipelines_raw/id2pipeline_path.pickle", "rb") as f:
    id2pipeline_path = pickle.load(f)
pipeline_path2id = {v: k for k, v in id2pipeline_path.items()}

In [3]:
df = pd.DataFrame()
dataset_id = []
architecture_id = []
hparam_id = []
pipeline_id = []
for p_id, p_path in id2pipeline_path.items():
    ds, arch, hp = p_path.split("/")[-3:]
    hp = hp.split(".")[0]
    dataset_id.append(ds)
    architecture_id.append(arch)
    hparam_id.append(hp)
    pipeline_id.append(p_id)

df["dataset_id"] = dataset_id
df["architecture_id"] = architecture_id
df["hparam_id"] = hparam_id
df["pipeline_id"] = pipeline_id

df1 = pd.read_csv("/home/cherniak/itmo_job/GAMLET/data/meta_features_and_fedot_pipelines_raw/old_train_task_pipe_comb.csv", index_col=0)
df2 = pd.read_csv("/home/cherniak/itmo_job/GAMLET/data/meta_features_and_fedot_pipelines_raw/old_test_task_pipe_comb.csv", index_col=0)
df_ = pd.concat([df1, df2])

df = df.merge(df_[["pipeline_id", "metric"]], on="pipeline_id")
df = df.dropna()

In [4]:
df

,dataset_id,architecture_id,hparam_id,pipeline_id,metric
0,sylvine_0,dbb8ea20-e8d0-47fc-9c2c-ae0910285475,0,0,-0.978000
1,sylvine_0,dbb8ea20-e8d0-47fc-9c2c-ae0910285475,1,1,-0.971000
2,sylvine_0,dbb8ea20-e8d0-47fc-9c2c-ae0910285475,2,2,-0.978000
3,sylvine_0,dbb8ea20-e8d0-47fc-9c2c-ae0910285475,3,3,-0.954000
4,sylvine_0,dbb8ea20-e8d0-47fc-9c2c-ae0910285475,4,4,-0.976000
...,...,...,...,...,...
2499543,vehicle_9,1a020599-1e3e-471c-bf4c-2ca777a5ab38,45,2500577,1.909581
2499544,vehicle_9,1a020599-1e3e-471c-bf4c-2ca777a5ab38,46,2500578,1.909581
2499545,vehicle_9,1a020599-1e3e-471c-bf4c-2ca777a5ab38,47,2500579,0.316544
2499546,vehicle_9,1a020599-1e3e-471c-bf4c-2ca777a5ab38,48,2500580,1.909581


In [5]:
train_dataset = df.groupby("dataset_id").apply(lambda x: x.sort_index().iloc[:int(0.8 * len(x))].reset_index(drop=True))
test_dataset = df.groupby("dataset_id").apply(lambda x: x.sort_index().iloc[int(0.8 * len(x)): ].reset_index(drop=True))

train_dataset = train_dataset.drop(columns=["dataset_id"])
test_dataset = test_dataset.drop(columns=["dataset_id"])

In [6]:
train_dataset.memory_usage(index=True, deep=True).sum() / 1024 / 1024

330.92105865478516

In [7]:
test_dataset.memory_usage(index=True, deep=True).sum() / 1024 / 1024

81.7867021560669

In [8]:
train_dataset

architecture_id hparam_id  \
dataset_id                                                           
australian_0 0      a77b3bfc-499e-45c4-82d7-e325b095a532         0   
             1      a77b3bfc-499e-45c4-82d7-e325b095a532         1   
             2      a77b3bfc-499e-45c4-82d7-e325b095a532         2   
             3      a77b3bfc-499e-45c4-82d7-e325b095a532         3   
             4      a77b3bfc-499e-45c4-82d7-e325b095a532         4   
...                                                  ...       ...   
vehicle_9    26464  d5f962c4-3042-4702-ab31-978ccfac5cf5        14   
             26465  d5f962c4-3042-4702-ab31-978ccfac5cf5        15   
             26466  d5f962c4-3042-4702-ab31-978ccfac5cf5        16   
             26467  d5f962c4-3042-4702-ab31-978ccfac5cf5        17   
             26468  d5f962c4-3042-4702-ab31-978ccfac5cf5        18   

                    pipeline_id     metric  
dataset_id                                  
australian_0 0           681507  -0.945000  
             1           681508  -0.943000  
             2           681509  -0.938000  
             3           681510  -0.948000  
             4           681511  -0.900000  
...                         ...        ...  
vehicle_9    26464      2493959  12.014551  
             26465      2493960   0.955868  
             26466      2493961   0.746583  
             26467      2493962   0.633949  
             26468      2493963   1.731304  

[1999616 rows x 4 columns]

In [9]:
test_dataset

architecture_id hparam_id  \
dataset_id                                                          
australian_0 0     26053c91-d52d-446b-8e8a-0a2a727b0379        19   
             1     26053c91-d52d-446b-8e8a-0a2a727b0379        20   
             2     26053c91-d52d-446b-8e8a-0a2a727b0379        21   
             3     26053c91-d52d-446b-8e8a-0a2a727b0379        22   
             4     26053c91-d52d-446b-8e8a-0a2a727b0379        23   
...                                                 ...       ...   
vehicle_9    6613  1a020599-1e3e-471c-bf4c-2ca777a5ab38        45   
             6614  1a020599-1e3e-471c-bf4c-2ca777a5ab38        46   
             6615  1a020599-1e3e-471c-bf4c-2ca777a5ab38        47   
             6616  1a020599-1e3e-471c-bf4c-2ca777a5ab38        48   
             6617  1a020599-1e3e-471c-bf4c-2ca777a5ab38        49   

                   pipeline_id    metric  
dataset_id                                
australian_0 0          693226 -0.920000  
             1          693227 -0.929000  
             2          693228 -0.924000  
             3          693229 -0.918000  
             4          693230 -0.930000  
...                        ...       ...  
vehicle_9    6613      2500577  1.909581  
             6614      2500578  1.909581  
             6615      2500579  0.316544  
             6616      2500580  1.909581  
             6617      2500581  0.804795  

[499932 rows x 4 columns]

In [10]:
dirname = "/home/cherniak/itmo_job/GAMLET/data/meta_features_and_fedot_pipelines_raw"

In [18]:
train_dataset.to_json("/home/cherniak/itmo_job/GAMLET/data/meta_features_and_fedot_pipelines_raw/train_task_pipe_comb.json",
orient="table")

test_dataset.to_json("/home/cherniak/itmo_job/GAMLET/data/meta_features_and_fedot_pipelines_raw/test_task_pipe_comb.json",
orient="table")

In [19]:
pd.read_json("/home/cherniak/itmo_job/GAMLET/data/meta_features_and_fedot_pipelines_raw/test_task_pipe_comb.json",
orient="table")

architecture_id hparam_id  \
dataset_id                                                          
australian_0 0     26053c91-d52d-446b-8e8a-0a2a727b0379        19   
             1     26053c91-d52d-446b-8e8a-0a2a727b0379        20   
             2     26053c91-d52d-446b-8e8a-0a2a727b0379        21   
             3     26053c91-d52d-446b-8e8a-0a2a727b0379        22   
             4     26053c91-d52d-446b-8e8a-0a2a727b0379        23   
...                                                 ...       ...   
vehicle_9    6613  1a020599-1e3e-471c-bf4c-2ca777a5ab38        45   
             6614  1a020599-1e3e-471c-bf4c-2ca777a5ab38        46   
             6615  1a020599-1e3e-471c-bf4c-2ca777a5ab38        47   
             6616  1a020599-1e3e-471c-bf4c-2ca777a5ab38        48   
             6617  1a020599-1e3e-471c-bf4c-2ca777a5ab38        49   

                   pipeline_id    metric  
dataset_id                                
australian_0 0          693226 -0.920000  
             1          693227 -0.929000  
             2          693228 -0.924000  
             3          693229 -0.918000  
             4          693230 -0.930000  
...                        ...       ...  
vehicle_9    6613      2500577  1.909581  
             6614      2500578  1.909581  
             6615      2500579  0.316544  
             6616      2500580  1.909581  
             6617      2500581  0.804795  

[499932 rows x 4 columns]

In [20]:
pd.read_json("/home/cherniak/itmo_job/GAMLET/data/meta_features_and_fedot_pipelines_raw/train_task_pipe_comb.json",
orient="table")

architecture_id hparam_id  \
dataset_id                                                           
australian_0 0      a77b3bfc-499e-45c4-82d7-e325b095a532         0   
             1      a77b3bfc-499e-45c4-82d7-e325b095a532         1   
             2      a77b3bfc-499e-45c4-82d7-e325b095a532         2   
             3      a77b3bfc-499e-45c4-82d7-e325b095a532         3   
             4      a77b3bfc-499e-45c4-82d7-e325b095a532         4   
...                                                  ...       ...   
vehicle_9    26464  d5f962c4-3042-4702-ab31-978ccfac5cf5        14   
             26465  d5f962c4-3042-4702-ab31-978ccfac5cf5        15   
             26466  d5f962c4-3042-4702-ab31-978ccfac5cf5        16   
             26467  d5f962c4-3042-4702-ab31-978ccfac5cf5        17   
             26468  d5f962c4-3042-4702-ab31-978ccfac5cf5        18   

                    pipeline_id     metric  
dataset_id                                  
australian_0 0           681507  -0.945000  
             1           681508  -0.943000  
             2           681509  -0.938000  
             3           681510  -0.948000  
             4           681511  -0.900000  
...                         ...        ...  
vehicle_9    26464      2493959  12.014551  
             26465      2493960   0.955868  
             26466      2493961   0.746583  
             26467      2493962   0.633949  
             26468      2493963   1.731304  

[1999616 rows x 4 columns]